# Part 6: AgentReview - Multi-Agent Peer Review 시스템 분석

**SNU AI Psychology Workshop - February 2026**

---

## 학습 목표

이 노트북은 **API 비용 없이** AgentReview의 구조와 개념을 학습합니다:

1. **Multi-Agent 시스템 아키텍처** 이해
2. **5단계 Peer Review 파이프라인** 분석
3. **리뷰어 페르소나** 설계 방법
4. **나만의 Agent 환경** 만들기 준비

---

## AgentReview (EMNLP 2024)

실제 **ICLR** peer review 프로세스를 LLM으로 시뮬레이션하는 프레임워크입니다.

**참고 자료:**
- [논문](https://arxiv.org/abs/2406.12708)
- [GitHub](https://github.com/Ahren09/AgentReview)
- [무료 데모 (HuggingFace)](https://huggingface.co/spaces/Ahren09/AgentReview) - 직접 실험해보세요!

---
## 1. 환경 설정

In [ ]:
# Cell 1: 환경 설정 (API 키 불필요)
import os
import sys
import json
from pathlib import Path

# 작업 디렉토리 설정
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    WORKSHOP_DIR = "/content/drive/MyDrive/aiworkshop_Feb2026/"
    IN_COLAB = True
    print("Colab 환경")
except ImportError:
    current_dir = os.getcwd()
    if current_dir.endswith('notebooks'):
        os.chdir('..')
    WORKSHOP_DIR = os.getcwd()
    IN_COLAB = False
    print("로컬 환경")

print(f"작업 폴더: {WORKSHOP_DIR}")

In [ ]:
# Cell 2: AgentReview 클론 (코드 분석용)
import subprocess

agentreview_path = os.path.join(WORKSHOP_DIR, "agentreview")

if not os.path.exists(agentreview_path):
    print("AgentReview 클론 중...")
    subprocess.run(["git", "clone", "https://github.com/ahren09/agentreview.git", agentreview_path], capture_output=True)
    print("AgentReview 클론 완료")
else:
    print("AgentReview 이미 존재")

# 모듈 경로 추가
sys.path.insert(0, agentreview_path)
print(f"\n코드 분석 준비 완료: {agentreview_path}")

---
## 2. Multi-Agent 아키텍처 이해하기

AgentReview는 다음 핵심 컴포넌트로 구성됩니다:

```
┌─────────────────────────────────────────────────────────┐
│                        Arena                            │
│  (게임 진행자: 턴 관리, 메시지 라우팅)                    │
├─────────────────────────────────────────────────────────┤
│                     Environment                         │
│  (규칙 정의: 5단계 파이프라인, 점수 체계)                 │
├─────────────────────────────────────────────────────────┤
│     Player 1      │     Player 2      │    Player 3     │
│   (Reviewer 1)    │   (Reviewer 2)    │   (Author)      │
│   ┌─────────┐     │   ┌─────────┐     │  ┌─────────┐    │
│   │ Backend │     │   │ Backend │     │  │ Backend │    │
│   │ (LLM)   │     │   │ (LLM)   │     │  │ (LLM)   │    │
│   └─────────┘     │   └─────────┘     │  └─────────┘    │
└─────────────────────────────────────────────────────────┘
```

- **Arena**: 전체 시뮬레이션을 orchestrate
- **Environment**: 게임 규칙과 상태 관리
- **Player**: 각 역할(리뷰어, 저자, AC)을 담당
- **Backend**: 실제 LLM API 호출 (OpenAI, HuggingFace 등)

In [ ]:
# Cell 3: 핵심 클래스 구조 분석

print("="*70)
print("AgentReview 핵심 클래스 구조")
print("="*70)

# 1. Agent 클래스
print("\n[1] Agent 클래스 (agentreview/agent.py)")
print("-"*50)
print("""
class Agent:
    '''기본 에이전트 - 이름과 역할 설명을 가짐'''
    
    def __init__(self, name, role_desc, ...):
        self.name = name           # 예: "Reviewer 1"
        self.role_desc = role_desc # 역할 설명 (시스템 프롬프트)
        self.backend = backend     # LLM 백엔드
""")

# 2. Player 클래스
print("\n[2] Player 클래스 (Agent 상속)")
print("-"*50)
print("""
class Player(Agent):
    '''게임에 참여하는 플레이어'''
    
    def act(self, observation):
        '''관찰을 받아 행동(텍스트) 생성'''
        # 1. 프롬프트 구성
        # 2. LLM 호출
        # 3. 응답 반환
        return self.backend.query(...)
""")

# 3. Environment 클래스
print("\n[3] Environment 클래스 (agentreview/environments/)")
print("-"*50)
print("""
class Environment:
    '''게임 환경 - 규칙과 상태 관리'''
    
    def step(self, player_name, action):
        '''한 턴 진행'''
        # 1. 행동 검증
        # 2. 상태 업데이트
        # 3. 다음 플레이어 결정
        return observations, rewards, done
""")

# 4. Arena 클래스
print("\n[4] Arena 클래스 (전체 진행)")
print("-"*50)
print("""
class Arena:
    '''시뮬레이션 진행자'''
    
    def run(self):
        while not done:
            # 1. 현재 플레이어 확인
            # 2. 관찰 제공
            # 3. 행동 요청
            # 4. 환경 업데이트
""")

---
## 3. 5단계 Peer Review 파이프라인

AgentReview의 핵심은 실제 학술 peer review 과정을 모방한 5단계 파이프라인입니다:

```
Phase 1          Phase 2              Phase 3           Phase 4         Phase 5
┌─────────┐     ┌─────────────┐      ┌───────────┐     ┌─────────┐    ┌──────────┐
│ Initial │ --> │   Author    │ -->  │ Reviewer- │ --> │  Meta-  │ -->│  Final   │
│ Review  │     │  Rebuttal   │      │AC Discuss │     │ Review  │    │ Decision │
└─────────┘     └─────────────┘      └───────────┘     └─────────┘    └──────────┘
  3 Reviewers     Author responds     Score updates      AC summary    Accept/Reject
  give scores     to each review      after discussion   + recommend
```

In [ ]:
# Cell 4: 5단계 파이프라인 상세 분석

pipeline_phases = {
    "Phase 1: Reviewer Assessment": {
        "참여자": ["Reviewer 1", "Reviewer 2", "Reviewer 3"],
        "목표": "각 리뷰어가 독립적으로 논문 평가",
        "출력": "초기 점수 (1-10) + 상세 리뷰",
        "특징": "리뷰어끼리 서로의 평가를 볼 수 없음"
    },
    "Phase 2: Author-Reviewer Discussion": {
        "참여자": ["Author", "Reviewers"],
        "목표": "저자가 리뷰에 대해 rebuttal 작성",
        "출력": "Rebuttal 문서",
        "특징": "저자는 모든 리뷰를 볼 수 있음"
    },
    "Phase 3: Reviewer-AC Discussion": {
        "참여자": ["Reviewers", "Area Chair (AC)"],
        "목표": "Rebuttal 기반으로 점수 재평가",
        "출력": "업데이트된 점수 + 의견",
        "특징": "리뷰어들이 서로의 리뷰와 점수를 볼 수 있음"
    },
    "Phase 4: Meta-Review": {
        "참여자": ["Area Chair (AC)"],
        "목표": "전체 리뷰 종합",
        "출력": "Meta-review + 추천 결정",
        "특징": "AC가 모든 정보를 종합하여 판단"
    },
    "Phase 5: Paper Decision": {
        "참여자": ["Area Chair (AC)"],
        "목표": "최종 Accept/Reject 결정",
        "출력": "Accept 또는 Reject",
        "특징": "점수, 리뷰, meta-review 모두 고려"
    }
}

print("="*70)
print("5단계 Peer Review 파이프라인 상세")
print("="*70)

for phase, details in pipeline_phases.items():
    print(f"\n{phase}")
    print("-"*50)
    for key, value in details.items():
        if isinstance(value, list):
            print(f"  {key}: {', '.join(value)}")
        else:
            print(f"  {key}: {value}")

---
## 4. 리뷰어 페르소나 시스템

AgentReview의 핵심 연구 질문: **리뷰어의 성향이 리뷰 결과에 어떤 영향을 미치는가?**

이를 위해 다양한 리뷰어 페르소나를 정의합니다:

In [ ]:
# Cell 5: 리뷰어 페르소나 속성 분석

persona_attributes = {
    "Intention (의도)": {
        "benign": "호의적 - 논문의 장점에 초점, 높은 점수 경향",
        "malicious": "악의적 - 논문의 단점에 초점, 낮은 점수 경향",
        "BASELINE": "중립 - 객관적 평가 시도"
    },
    "Knowledgeability (전문성)": {
        "knowledgeable": "전문가 - 해당 분야 깊은 이해, 기술적 세부사항 평가",
        "unknowledgeable": "비전문가 - 표면적 평가, 기술적 오류 가능",
        "BASELINE": "보통 수준의 전문성"
    },
    "Commitment (책임감)": {
        "responsible": "성실 - 꼼꼼한 리뷰, 건설적 피드백",
        "irresponsible": "불성실 - 짧은 리뷰, 피상적 코멘트",
        "BASELINE": "보통 수준의 성실성"
    },
    "Author Awareness (저자 인지)": {
        "authors_are_famous": "유명 저자로 인식 - 후광 효과 가능",
        "authors_are_unfamous": "무명 저자로 인식 - 편견 가능",
        "(기본)": "저자 정보 모름 (블라인드 리뷰)"
    }
}

print("="*70)
print("리뷰어 페르소나 속성")
print("="*70)

for category, attributes in persona_attributes.items():
    print(f"\n{category}")
    print("-"*50)
    for attr, desc in attributes.items():
        print(f"  {attr:25} : {desc}")

In [ ]:
# Cell 6: 실제 페르소나 프롬프트 분석

# AgentReview의 role_descriptions.py에서 가져온 실제 프롬프트 예시
persona_prompts = {
    "benign": """
You are a benevolent reviewer who wants to help authors improve their work.
You tend to focus on the positive aspects of papers and give constructive feedback.
You are more likely to give higher scores to papers that show potential.
""",
    "malicious": """
You are a harsh reviewer who is very critical of submitted papers.
You tend to focus on the negative aspects and potential flaws.
You set very high standards and are more likely to reject papers.
""",
    "knowledgeable": """
You are an expert in this field with deep understanding of the technical details.
You can evaluate the novelty, correctness, and significance of the contributions.
You provide detailed technical feedback based on your expertise.
""",
    "unknowledgeable": """
You are not very familiar with this specific research area.
You may miss some technical nuances and focus more on presentation quality.
Your evaluation may be more surface-level.
""",
    "responsible": """
You are a dedicated reviewer who takes the review process seriously.
You provide thorough, detailed feedback with specific examples.
You suggest concrete improvements and are open to discussion.
""",
    "irresponsible": """
You are a busy reviewer who doesn't have much time for this paper.
Your reviews tend to be brief and lack specific details.
You may not engage deeply with the technical content.
"""
}

print("="*70)
print("페르소나별 시스템 프롬프트 예시")
print("="*70)

for persona, prompt in persona_prompts.items():
    print(f"\n[{persona}]")
    print("-"*40)
    print(prompt.strip())

In [ ]:
# Cell 7: 페르소나 조합 실습

print("="*70)
print("페르소나 조합 예시")
print("="*70)

# 조합 방법: 언더스코어(_)로 연결
example_combinations = [
    ("knowledgeable_responsible", "전문가이면서 성실한 리뷰어 (이상적)"),
    ("malicious_unknowledgeable", "악의적이면서 비전문가 (최악)"),
    ("benign_irresponsible", "호의적이지만 불성실 (짧은 긍정 리뷰)"),
    ("knowledgeable_malicious", "전문가이지만 악의적 (날카로운 비판)"),
    ("authors_are_famous", "유명 저자라고 인식 (후광 효과)"),
]

print("\n조합 방법: 속성들을 '_'로 연결")
print("-"*50)
for combo, desc in example_combinations:
    print(f"  '{combo}'")
    print(f"     -> {desc}\n")

---
## 5. 실험 설정 구성하기

AgentReview 실험을 위한 설정 구조를 이해합니다.

In [ ]:
# Cell 8: 실험 설정 템플릿

# 기본 설정 템플릿
experiment_settings = {
    "baseline": {
        "name": "BASELINE",
        "description": "모든 참여자가 중립적",
        "reviewer": ["BASELINE", "BASELINE", "BASELINE"],
        "AC": ["BASELINE"],
        "author": ["BASELINE"],
        "global_settings": {
            "provides_numeric_rating": ["reviewer", "ac"],
            "persons_aware_of_authors_identities": []
        }
    },
    "malicious_reviewer": {
        "name": "MALICIOUS",
        "description": "악의적 리뷰어 1명 포함",
        "reviewer": ["malicious", "BASELINE", "BASELINE"],
        "AC": ["BASELINE"],
        "author": ["BASELINE"],
        "global_settings": {
            "provides_numeric_rating": ["reviewer", "ac"],
            "persons_aware_of_authors_identities": []
        }
    },
    "expert_panel": {
        "name": "EXPERT",
        "description": "전문가 패널",
        "reviewer": ["knowledgeable_responsible", "knowledgeable", "knowledgeable_benign"],
        "AC": ["knowledgeable"],
        "author": ["BASELINE"],
        "global_settings": {
            "provides_numeric_rating": ["reviewer", "ac"],
            "persons_aware_of_authors_identities": []
        }
    },
    "fame_bias_test": {
        "name": "FAME_BIAS",
        "description": "저자 유명도 편향 테스트",
        "reviewer": ["authors_are_famous", "authors_are_unfamous", "BASELINE"],
        "AC": ["BASELINE"],
        "author": ["BASELINE"],
        "global_settings": {
            "provides_numeric_rating": ["reviewer", "ac"],
            "persons_aware_of_authors_identities": ["Reviewer 1", "Reviewer 2"]
        }
    }
}

print("="*70)
print("실험 설정 템플릿")
print("="*70)

for key, setting in experiment_settings.items():
    print(f"\n[{setting['name']}] {setting['description']}")
    print("-"*50)
    print(f"  Reviewers: {setting['reviewer']}")
    print(f"  AC: {setting['AC']}")
    print(f"  Author: {setting['author']}")

---
## 6. DIY: 나만의 리뷰어 조합 만들기

이제 직접 리뷰어 설정을 만들어봅시다!

In [ ]:
# Cell 9: DIY - 나만의 실험 설정

# TODO: 아래 설정을 수정해보세요!

my_experiment = {
    "name": "MY_EXPERIMENT",  # <- 실험 이름
    "description": "나만의 실험 설정",  # <- 설명
    
    # 3명의 리뷰어 설정
    # 사용 가능한 속성: benign, malicious, knowledgeable, unknowledgeable,
    #                  responsible, irresponsible, authors_are_famous, authors_are_unfamous
    # 조합: "knowledgeable_responsible", "malicious_unknowledgeable" 등
    "reviewer": [
        "BASELINE",      # Reviewer 1 - 수정해보세요!
        "BASELINE",      # Reviewer 2 - 수정해보세요!
        "BASELINE",      # Reviewer 3 - 수정해보세요!
    ],
    
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    
    "global_settings": {
        "provides_numeric_rating": ["reviewer", "ac"],
        "persons_aware_of_authors_identities": []  # 블라인드 리뷰
    }
}

print("="*70)
print("나의 실험 설정")
print("="*70)
print(f"\n실험명: {my_experiment['name']}")
print(f"설명: {my_experiment['description']}")
print(f"\nReviewers:")
for i, r in enumerate(my_experiment['reviewer'], 1):
    print(f"  Reviewer {i}: {r}")
print(f"\nAC: {my_experiment['AC'][0]}")
print(f"Author: {my_experiment['author'][0]}")

In [ ]:
# Cell 10: 실험 아이디어

research_questions = [
    {
        "question": "악의적 리뷰어가 1명 있으면 결과가 얼마나 달라질까?",
        "setting": ["malicious", "BASELINE", "BASELINE"],
        "comparison": ["BASELINE", "BASELINE", "BASELINE"]
    },
    {
        "question": "전문가 리뷰어가 비전문가보다 더 공정할까?",
        "setting": ["knowledgeable_responsible", "knowledgeable_responsible", "knowledgeable_responsible"],
        "comparison": ["unknowledgeable", "unknowledgeable", "unknowledgeable"]
    },
    {
        "question": "유명 저자 효과는 얼마나 클까?",
        "setting": ["authors_are_famous", "authors_are_famous", "authors_are_famous"],
        "comparison": ["authors_are_unfamous", "authors_are_unfamous", "authors_are_unfamous"]
    },
    {
        "question": "불성실한 리뷰어도 rebuttal 후 점수를 바꿀까?",
        "setting": ["irresponsible", "irresponsible", "irresponsible"],
        "comparison": ["responsible", "responsible", "responsible"]
    },
]

print("="*70)
print("실험 아이디어 (연구 질문)")
print("="*70)

for i, rq in enumerate(research_questions, 1):
    print(f"\n{i}. {rq['question']}")
    print(f"   실험군: {rq['setting']}")
    print(f"   대조군: {rq['comparison']}")

---
## 7. 커스텀 Environment 만들기

AgentReview의 구조를 응용하여 나만의 Multi-Agent 환경을 만들 수 있습니다.

In [ ]:
# Cell 11: 커스텀 Environment 템플릿

custom_env_template = '''
# my_custom_environment.py

from agentreview.environments import Environment

class MyCustomEnvironment(Environment):
    """나만의 Multi-Agent 환경"""
    
    type_name = "my_custom_env"
    
    def __init__(self, player_names, **kwargs):
        super().__init__(player_names=player_names, **kwargs)
        
        # 환경 상태 초기화
        self.current_phase = 0
        self.phases = ["Phase 1", "Phase 2", "Phase 3"]
        self.message_history = []
    
    def get_next_player(self):
        """다음 행동할 플레이어 결정"""
        # 현재 phase에 따라 플레이어 결정
        if self.current_phase == 0:
            return self.player_names[0]  # 첫 번째 플레이어
        elif self.current_phase == 1:
            return self.player_names[1]  # 두 번째 플레이어
        else:
            return None  # 종료
    
    def step(self, player_name, action):
        """한 턴 진행"""
        # 1. 메시지 저장
        self.message_history.append({
            "player": player_name,
            "action": action,
            "phase": self.phases[self.current_phase]
        })
        
        # 2. 다음 phase로 이동
        self.current_phase += 1
        
        # 3. 종료 조건 확인
        done = self.current_phase >= len(self.phases)
        
        # 4. 다음 플레이어에게 전달할 관찰
        observations = {
            p: self._get_observation_for_player(p)
            for p in self.player_names
        }
        
        return observations, {}, done
    
    def _get_observation_for_player(self, player_name):
        """플레이어별 관찰 생성"""
        # 각 플레이어가 볼 수 있는 정보 결정
        return f"Current phase: {self.phases[min(self.current_phase, len(self.phases)-1)]}"
'''

print("="*70)
print("커스텀 Environment 템플릿")
print("="*70)
print(custom_env_template)

In [ ]:
# Cell 12: 응용 아이디어

application_ideas = [
    {
        "name": "논문 작성 도우미",
        "players": ["Writer", "Critic", "Editor"],
        "phases": ["초안 작성", "비평", "수정", "최종 검토"],
        "description": "여러 관점에서 논문을 개선하는 시스템"
    },
    {
        "name": "면접 시뮬레이터",
        "players": ["Interviewer", "Candidate", "Observer"],
        "phases": ["자기소개", "기술질문", "행동질문", "피드백"],
        "description": "면접 연습 및 피드백 시스템"
    },
    {
        "name": "디버깅 팀",
        "players": ["Developer", "Tester", "Reviewer"],
        "phases": ["코드 작성", "테스트", "리뷰", "수정"],
        "description": "코드 품질 개선 시스템"
    },
    {
        "name": "연구 아이디어 평가",
        "players": ["Proposer", "Devil's Advocate", "Supporter", "Judge"],
        "phases": ["아이디어 제안", "반론", "옹호", "평가"],
        "description": "연구 아이디어의 강점과 약점 분석"
    },
]

print("="*70)
print("Multi-Agent 시스템 응용 아이디어")
print("="*70)

for idea in application_ideas:
    print(f"\n[{idea['name']}]")
    print(f"  설명: {idea['description']}")
    print(f"  참여자: {' -> '.join(idea['players'])}")
    print(f"  단계: {' -> '.join(idea['phases'])}")

---
## 8. 실제 실행 (선택사항)

**주의: API 비용이 발생합니다!**

실제로 AgentReview를 실행하려면:
1. OpenAI API 키가 필요합니다
2. 각 실행당 약 $0.5-2 정도의 비용이 발생합니다
3. 실행 시간은 5-15분 정도 소요됩니다

**무료 대안:**
- [HuggingFace Demo](https://huggingface.co/spaces/Ahren09/AgentReview)에서 무료로 실험해볼 수 있습니다!

In [ ]:
# Cell 13: [선택사항] 실제 실행 코드
# 
# 아래 코드는 주석 처리되어 있습니다.
# 실행하려면 주석을 해제하고 OpenAI API 키를 설정하세요.
# 
# 비용: 약 $0.5-2 per run
# 시간: 약 5-15분

'''
# 1. API 키 설정
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    raise ValueError("OpenAI API 키가 필요합니다!")

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

# 2. AgentReview 모듈 로딩
import sys
sys.path.insert(0, "agentreview")

from agentreview.environments import PaperReview
from agentreview.paper_review_arena import PaperReviewArena
from agentreview.paper_review_settings import get_experiment_settings
from agentreview.utility.experiment_utils import initialize_players
from agentreview.utility.utils import get_paper_decision_mapping
from agentreview.arguments import parse_args

# 3. 설정
sys.argv = [
    'notebook',
    '--conference', 'ICLR2022',
    '--model_name', 'gpt-4o',
    '--data_dir', './data',
    '--experiment_name', my_experiment['name']
]

args = parse_args()
args.task = "paper_review"

# 4. 실행
paper_id2decision, _ = get_paper_decision_mapping(args.data_dir, args.conference)
paper_id = 284  # 샘플 논문 ID

experiment_setting = get_experiment_settings(
    paper_id=paper_id,
    paper_decision=paper_id2decision[paper_id],
    setting=my_experiment
)

players = initialize_players(experiment_setting=experiment_setting, args=args)
env = PaperReview(
    player_names=[p.name for p in players],
    paper_decision=paper_id2decision[paper_id],
    paper_id=paper_id,
    args=args,
    experiment_setting=experiment_setting
)

arena = PaperReviewArena(players=players, environment=env, args=args)
arena.launch_cli(interactive=False)

print("실행 완료!")
'''

print("="*70)
print("실제 실행 코드 (주석 처리됨)")
print("="*70)
print("\n위 코드의 주석을 해제하면 AgentReview를 실행할 수 있습니다.")
print("\n무료 대안: HuggingFace Demo")
print("https://huggingface.co/spaces/Ahren09/AgentReview")

---
## 9. 토론 주제

### 연구 질문

1. **리뷰어 편향의 영향**
   - 악의적 리뷰어 1명이 전체 결과에 미치는 영향은?
   - 유명 저자 효과는 얼마나 클까?

2. **5단계 파이프라인의 가치**
   - Rebuttal이 점수 변화에 얼마나 효과적인가?
   - AC 토론이 편향을 줄이는가?

3. **실제 활용**
   - 논문 투고 전 사전 테스트로 사용할 수 있을까?
   - 리뷰어 배정 최적화에 활용할 수 있을까?

### 나만의 Agent 만들기

1. **어떤 분야에 Multi-Agent 시스템을 적용하고 싶은가?**
   - 논문 작성? 코드 리뷰? 면접 연습?

2. **어떤 역할(페르소나)이 필요한가?**
   - 비평가? 지지자? 중재자?

3. **어떤 단계(파이프라인)가 필요한가?**
   - 몇 단계? 각 단계에서 무슨 일이 일어나는가?

---

## 다음 단계

1. **HuggingFace Demo 체험**: [AgentReview Demo](https://huggingface.co/spaces/Ahren09/AgentReview)
2. **코드 분석**: `agentreview/` 폴더의 소스코드 탐색
3. **커스텀 환경 설계**: 위의 템플릿을 기반으로 나만의 환경 설계

In [ ]:
# Cell 14: 워크숍 완료!

print("="*70)
print("Part 6 완료!")
print("="*70)
print("\n학습한 내용:")
print("  1. Multi-Agent 시스템 아키텍처 (Arena, Environment, Player, Backend)")
print("  2. 5단계 Peer Review 파이프라인")
print("  3. 리뷰어 페르소나 설계 방법")
print("  4. 커스텀 Environment 만들기")
print("\n다음 단계:")
print("  - HuggingFace Demo에서 직접 실험해보기")
print("  - 나만의 Multi-Agent 시스템 설계하기")
print("\n무료 데모: https://huggingface.co/spaces/Ahren09/AgentReview")